In [1]:
import numpy as np
import pandas as pd

In [2]:
!pwd

/Users/jamesreno/Documents/Documents - James’s MacBook Air/NYCDSA/Python/Python Project/Meetup_Data


In [3]:
rsvps = pd.read_csv('rsvps.csv')
meta_members = pd.read_csv('meta-members.csv')
meta_groups = pd.read_csv('meta-groups.csv')
meta_events = pd.read_csv('meta-events.csv')
m_to_g_edges = pd.read_csv('member-to-group-edges.csv')
member_edges = pd.read_csv('member-edges.csv')
group_edges = pd.read_csv('group-edges.csv')

Data Organization
Below are the data sets retrieved from: https://www.kaggle.com/datasets/stkbailey/nashville-meetup?select=meta-groups.csv
on 04/19/2022

Defined their assumed definitions and links between data sets.

rsvps: (list of rsvps with member, event, and group)
has two indices - probably useless
event_id - unique id might be used to join to the meta_events
member_id - unique id might be used to join to: meta_members, m_to_g_edges, member_edges
group_id - unique id might be used to join to: meta_groups, m_to_g_edges, group_edges

meta_members: (list of members)
member_id - unique id might be used to join to m_to_g_edges, member_edges, rsvps
name - name of member
?hometown- either where they live or where they grew up
?city - where they joined meetup?
?state - where they are attending meetups?
lat - lattitude
lon - longitude

meta_groups: (list of groups)
group_id - unique id might be used to join to: rsvps, meta_events, m_to_g_edges, group_edges
group_name - group name
num_members - number of members
category_id - unique category id
category_name - broad category of the type of group
?organizer_id - member id of the organizer(s) or admin(s) ??? Need to confirm
group_urlname - url within the meetup.com space

meta_events: (list of events)
event_id - unique id might be used to join to: rsvps
group_id - unique id might be used to join to: rsvps, meta_groups, m_to_g_edges, group_edges
name - event name
time - Nashville time of event start time

m_to_g_edges: (member to group connections)
member_id - unique id might be used to join to: rsvps, meta_members, meta_events, member_edges
group_id - unique id might be used to join to: rsvps, meta_groups, meta_events, group_edges
weight - ?

member_edges: (data used to show connection between members)
member1 - unique member id
member2 - unique member id
weight - ?

group_edges: (data used to show connections between groups)
group1 - unique group id
group2 - unique group id
weight - ?

In [20]:
member_edges.head()

,Unnamed: 0,member1,member2,weight
0,0,198737924,220654721,1
1,1,198737924,208201738,1
2,2,198737924,88664332,1
3,3,198737924,8640526,1
4,4,198737924,56356372,1


In [25]:
rsvps.head()

,Unnamed: 0,event_id,member_id,group_id
0,0,243930425,6770985,26140018
1,1,244208851,234724627,25604533
2,2,pxlktnywnbfb,199563568,25973656
3,3,pxlktnywnbfb,208888939,25973656
4,4,244444551,1720391,18571126


In [26]:
rsvps.describe()

,Unnamed: 0,member_id,group_id
count,126813.000000,1.268130e+05,1.268130e+05
mean,63406.000000,1.313540e+08,1.158637e+07
std,36607.904181,8.368705e+07,8.092142e+06
min,0.000000,2.069000e+03,4.709400e+04
25%,31703.000000,3.859148e+07,1.763190e+06
50%,63406.000000,1.753005e+08,1.177650e+07
75%,95109.000000,2.036641e+08,1.895583e+07
max,126812.000000,2.400034e+08,2.629659e+07


In [29]:
rsvps.dtypes

Unnamed: 0     int64
event_id      object
member_id      int64
group_id       int64
dtype: object

In [31]:
len(rsvps['event_id'].unique())

19031

In [47]:
rsvps[rsvps[['event_id','member_id','group_id']].isna().all(axis=1)]

,Unnamed: 0,event_id,member_id,group_id


In [54]:
rsvps[rsvps['group_id'].isnull()]

,Unnamed: 0,event_id,member_id,group_id


In [75]:
def drop_unnamed(df):
    #checking if 'Unnamed: 0' exists as a column name, if so drop and reassign
    
    if 'Unnamed: 0' in df.columns:
        return df.drop(labels='Unnamed: 0',axis=1)
    

In [76]:
rsvps=drop_unnamed(rsvps)
#rsvps.drop(labels='Unnamed: 0',axis=1)

In [79]:
for df in [rsvps,meta_members,meta_groups,meta_events,m_to_g_edges,member_edges,group_edges]:
    df = drop_unnamed(df)

In [80]:
meta_members.head()

,member_id,name,hometown,city,state,lat,lon
0,2069,Wesley Duffee-Braun,Brentwood,Brentwood,TN,36.00,-86.79
1,8386,Tim,Nashville,Nashville,TN,36.07,-86.78
2,9205,Brenda,Brentwood,Brentwood,TN,36.00,-86.79
3,17903,Steve,NaN,Nashville,TN,36.13,-86.80
4,20418,Andrea Reynolds,"Huntington, WV",Nashville,TN,36.17,-86.72


In [83]:
meta_members.isna().sum()

member_id        0
name             0
hometown     19664
city             0
state           94
lat              0
lon              0
dtype: int64

In [87]:
len(meta_members['member_id'].unique())

24591

In [86]:
len(meta_members)

24591

In [89]:
merged_df = meta_members.merge(rsvps, how='inner', on='member_id')

In [ ]:
# am going to count the number of events attended and number organized etc
merged_df.groupby(member_id)